<a href="https://colab.research.google.com/github/leehee-barak/data-analysis-projects/blob/master/02_Regression_%26_Regularization_Exercise_Leehee_Barak_Adir_golan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# masheu aher
# masheu aher 2

# Ydata Data Science School
## Linear Regression & Regularization Exercise.


## Outline
In this exercise you will learn the following topics:

1. Refresher on how linear regression is solved in batch and in Gradient Descent
2. Implementation of Ridge Regression
3. Comparing Ridge, Lasso and vanila Linear Regression on a dataset

## Package import and data reading
boston housing dataframe = bhdf

ecomerce costumers dataframe = ecdf

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold, train_test_split, ShuffleSplit
from scipy.stats import ttest_rel, t

In [ ]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

## Refresher on Ordinary Least Square (OLS) aka Linear Regeression

### Lecture Note

In Matrix notation, the matrix $X$ is of dimensions $n \times p$ where each row is an example and each column is a feature dimension.

Similarily, $y$ is of dimension $n \times 1$ and $w$ is of dimensions $p \times 1$.

The model is $\hat{y}=X\cdot w$ where we assume for simplicity that $X$'s first columns equals to 1 (one padding), to account for the bias term.

Our objective is to optimize the loss $L$ defines as resiudal sum of squares (RSS):

$L_{RSS}=\frac{1}{N}\left\Vert Xw-y \right\Vert^2$ (notice that in matrix notation this means summing over all examples, so $L$ is scalar.)

To find the optimal $w$ one needs to derive the loss with respect to $w$.

$\frac{\partial{L_{RSS}}}{\partial{w}}=\frac{2}{N}X^T(Xw-y)$ (to see why, read about [matrix derivatives](http://www.gatsby.ucl.ac.uk/teaching/courses/sntn/sntn-2017/resources/Matrix_derivatives_cribsheet.pdf) or see class notes )

Thus, the gardient descent solution is $w'=w-\alpha \frac{2}{N}X^T(Xw-y)$.

Solving $\frac{\partial{L_{RSS}}}{\partial{w}}=0$ for $w$ one can also get analytical solution:

$w_{OLS}=(X^TX)^{-1}X^Ty$

The first term, $(X^TX)^{-1}X^T$ is also called the pseudo inverse of $X$.

See [lecture note from Stanford](https://web.stanford.edu/~mrosenfe/soc_meth_proj3/matrix_OLS_NYU_notes.pdf) for more details.


## Exercise 1 - Ordinary Least Square
* Get the boston housing dataset https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

* What is $p$? what is $n$ in the above notation? hint: [shape](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.ndarray.shape.html)

* write a model `OrdinaryLinearRegression` which has a property $w$ and 3 methods: `fit`, `predict` and `score` (which returns the MSE on a given sample set). Hint: use [numpy.linalg.pinv](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.linalg.pinv.html) to be more efficient.

* Fit the model. What is the training MSE?

* Plot a scatter plot where on x-axis plot $Y$ and in the y-axis $\hat{Y}_{OLS}$

* Split the data to 75% train and 25% test 20 times. What is the average MSE now for train and test? Hint: use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) or [ShuffleSplit](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html).

* Use a t-test to proove that the MSE for training is significantly smaller than for testing. What is the p-value? Hint: use [scipy.stats.ttest_rel](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.ttest_rel.html).

* Write a new class `OrdinaryLinearRegressionGradientDescent` which inherits from `OrdinaryLinearRegression` and solves the problem using gradinet descent. The class should get as a parameter the learning rate and number of iterations. Plot the class convergance. What is the effect of learning rate? How would you find number of iteration automatically? Note: Gradient Descent does not work well when features are not scaled evenly (why?!). Be sure to normalize your features first.

* The following parameters are optional (not mandatory to use):
    * early_stop - True / False boolean to indicate to stop running when loss stops decaying and False to continue.
    * verbose- True/False boolean to turn on / off logging, e.g. print details like iteration number and loss (https://en.wikipedia.org/wiki/Verbose_mode)
    * track_loss - True / False boolean when to save loss results to present later in learning curve graphs

###Code

In [ ]:
# @title OLS
# testo
# * write a model `Ols` which has a propoery $w$ and 3 methods: `fit`, `predict` and `score`.? hint: use [numpy.linalg.pinv](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.linalg.pinv.html) to be more efficient.

class Ols(object):
  def __init__(self):
    self.w = None
    self._estimator_type = 'classifier'

  @staticmethod
  def pad(X):
    # pads the matrix X with a ones vector
    return np.insert(X, 0, np.ones(X.shape[0]), axis = 1)

  def fit(self, X, Y):
    #remeber pad with 1 before fitting
    X = self.pad(X)
    Xpinv = np.linalg.pinv(X)
    self.w = np.dot(Xpinv, Y)
    return

  def _fit(self, X, Y):
    # optional to use this
    pass

  def predict(self, X):
    #return wx
    X = self.pad(X)
    return np.dot(X, self.w)
    pass

  def _predict(self, X):
    # optional to use this
    pass

  def score(self, X, Y):
    #return MSE
    ypred = self.predict(X)
    return np.mean(((Y - ypred)**2))

  def get_params(self, deep: bool = True):
    return dict()


In [ ]:
# @title GD
# Write a new class OlsGd which solves the problem using gradinet descent.
# The class should get as a parameter the learning rate and number of iteration.
# Plot the loss convergance. for each alpha, learning rate plot the MSE with respect to number of iterations.
# What is the effect of learning rate?
# How would you find number of iteration automatically?
# Note: Gradient Descent does not work well when features are not scaled evenly (why?!). Be sure to normalize your feature first.
class Normalizer():
  def __init__(self):
    pass

  def fit(self, X):
    pass

  def predict(self, X):
    #apply normalization
    pass

class OlsGd(Ols):

  def __init__(self, learning_rate=.05,
               num_iteration=1000,
               normalize=True,
               early_stop=True,
               verbose=True):

    super(OlsGd, self).__init__()
    self.learning_rate = learning_rate
    self.num_iteration = num_iteration
    self.early_stop = early_stop
    self.normalize = normalize
    self.normalizer = Normalizer()
    self.verbose = verbose

  def _fit(self, X, Y, reset=True, track_loss=True):
    #remeber to normalize the data before starting
    pass

  def _predict(self, X):
    #remeber to normalize the data before starting
    pass

  def _step(self, X, Y):
    # use w update for gradient descent
    pass

###Playground

In [ ]:
np.linalg.pinv?

In [ ]:
n = 10
p = 5
X = np.round(np.random.rand(n,p), 3)
X

array([[0.333, 0.852, 0.645, 0.314, 0.773],
       [0.741, 0.338, 0.155, 0.518, 0.647],
       [0.441, 0.498, 0.299, 0.506, 0.046],
       [0.704, 0.487, 0.507, 0.208, 0.882],
       [0.71 , 0.49 , 0.877, 0.212, 0.814],
       [0.301, 0.032, 0.385, 0.118, 0.317],
       [0.88 , 0.903, 0.702, 0.509, 0.28 ],
       [0.878, 0.584, 0.828, 0.072, 0.266],
       [0.104, 0.688, 0.317, 0.003, 0.467],
       [0.434, 0.259, 0.237, 0.169, 0.653]])

In [ ]:
Y = np.random.randint(0, 2, (n,1))
Y.T

array([[1, 1, 1, 1, 1, 0, 0, 0, 1, 1]])

In [ ]:
myols = Ols()
myols.fit(X,Y)
myols.w.T

array([[ 0.43148833, -0.59228774,  0.34389606, -0.65185934,  0.63054053,
         1.12028852]])

In [ ]:
ypred = myols.predict(X)
np.round(ypred).T

array([[ 1.,  1.,  1.,  1.,  1.,  0.,  0., -0.,  1.,  1.]])

In [ ]:
myols.score(X, Y)

0.07565688415196015

In [ ]:
(np.round(ypred, 0) == Y).T

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])

In [ ]:
def pad(X):
    # pads the matrix X with a ones vector
    return np.insert(X, 0, np.ones(X.shape[0]), axis = 1)

In [ ]:
# X1 = pad(X)
# XT = np.transpose(X1)
# XTX = np.dot(XT, X1)
# XTXpinv = np.linalg.pinv(XTX)
# extw = np.dot(XTXpinv, np.dot(XT, Y))
# extw
# yext = np.dot(pad(X), extw)
# yext

In [ ]:
dataols = Ols()
dataols.fit(data, target)

In [ ]:
tpred = dataols.predict(data)
# ratio = (np.round(tpred) == np.round(target))
# ratio.sum()/ratio.shape[0]

In [ ]:
dataols.score(data, target)

21.894831181729206

In [ ]:
# model = Ols()
# ss = ShuffleSplit(n_splits=20, test_size=0.25, random_state=42)
# cross_val_score(model, data, target, cv=ss)

array([22.09869483, 17.53710121, 32.80613615, 18.30791922, 20.27597662,
       28.63216657, 16.32458011, 28.05274015, 35.04182222, 29.26598993,
       18.46827486, 24.96668596, 22.95527308, 31.91913652, 18.67142826,
       30.51444137, 28.18756248, 38.67639356, 23.24880593, 24.19951726])

In [ ]:
model = Ols()
scores = np.zeros((2,20))
stat_dict = {'Train MSE': [], 'Test MSE': [], 't_statistic': [], 'p_value': []}

for i in range(scores.shape[1]):
  X_train, X_test, Y_train, Y_test = train_test_split(data, target, test_size=0.25, random_state=2)
  model.fit(X_train, Y_train)
  scores[0, i] = model.score(X_train, Y_train)
  scores[1, i] = model.score(X_test, Y_test)
stat_dict['Train MSE'].append(round(np.mean(scores, axis = 1)[0], 2))
stat_dict['Test MSE'].append(round(np.mean(scores, axis = 1)[1], 2))
stat_dict['t_statistic'].append(np.round(ttest_rel(scores[0], scores[1])[0], 2))
stat_dict['p_value'].append(round(ttest_rel(scores[0], scores[1])[1], 2))
single_20 = pd.DataFrame(stat_dict)
single_20

/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


,Train MSE,Test MSE,t_statistic,p_value
0,22.04,22.16,-inf,0.0


In [ ]:
model = Ols()
n_runs = 10000
stat_dict = {'Train MSE': [], 'Test MSE': [], 't_statistic': [], 'p_value': []}
for j in range(n_runs):
  scores = np.zeros((2,20))
  for i in range(scores.shape[1]):
    X_train, X_test, Y_train, Y_test = train_test_split(data, target, test_size=0.25, random_state=(i))
    model.fit(X_train, Y_train)
    scores[0, i] = model.score(X_train, Y_train)
    scores[1, i] = model.score(X_test, Y_test)
  stat_dict['Train MSE'].append(round(np.mean(scores, axis = 1)[0], 2))
  stat_dict['Test MSE'].append(round(np.mean(scores, axis = 1)[1], 2))
  stat_dict['t_statistic'].append(np.round(ttest_rel(scores[0], scores[1])[0], 2))
  stat_dict['p_value'].append(round(ttest_rel(scores[0], scores[1])[1], 2))

stats10kpr = pd.DataFrame(stat_dict)
stats10kpr.describe().head(3)

,Train MSE,Test MSE,t_statistic,p_value
count,10000.00,1.000000e+04,1.000000e+04,10000.00
mean,21.23,2.515000e+01,-3.020000e+00,0.01
std,0.00,3.552891e-15,8.882228e-16,0.00


In [ ]:
stats10k.describe()

,Train MSE,Test MSE,t_statistic,p_value
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,21.590690,24.082385,-1.670886,0.214245
std,0.356417,1.155517,0.998413,0.244969
min,20.270000,20.330000,-6.500000,0.000000
25%,21.350000,23.290000,-2.290000,0.030000
50%,21.600000,24.040000,-1.660000,0.110000
75%,21.830000,24.840000,-1.030000,0.310000
max,22.790000,28.550000,2.530000,1.000000


In [ ]:
stats10k[stats10k.p_value < 0.05].shape

(3007, 4)

In [ ]:
stats10kr.describe() # random_state=(i+j)

,Train MSE,Test MSE,t_statistic,p_value
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,21.585249,24.099355,-1.685145,0.213835
std,0.368346,1.199347,1.019992,0.249911
min,20.240000,20.230000,-5.360000,0.000000
25%,21.350000,23.280000,-2.360000,0.030000
50%,21.595000,24.070000,-1.680000,0.110000
75%,21.830000,24.840000,-1.030000,0.310000
max,22.800000,28.540000,2.550000,1.000000


In [ ]:
stats10kpr.describe() # random_state=(i)

,Train MSE,Test MSE,t_statistic,p_value
count,10000.00,1.000000e+04,1.000000e+04,10000.00
mean,21.23,2.515000e+01,-3.020000e+00,0.01
std,0.00,3.552891e-15,8.882228e-16,0.00
min,21.23,2.515000e+01,-3.020000e+00,0.01
25%,21.23,2.515000e+01,-3.020000e+00,0.01
50%,21.23,2.515000e+01,-3.020000e+00,0.01
75%,21.23,2.515000e+01,-3.020000e+00,0.01
max,21.23,2.515000e+01,-3.020000e+00,0.01


###End playgroud

###Answers


*   The number of samples, **n**, in the Boston Housing dataset is ***506***
*   The number of features, **p**, in the Boston Housing dataset is ***13***
*   The training MSE is ***21.89***
*   The average average MSEs of 20 splits of 25%-75%:
    *   Train: 21.56
    *   Test: 24.12
*   The p-value is
*   The effect GD has on learning rate is
*   Finding the number of iterations automatically...


In [ ]:
# Plot a scatter plot:
import matplotlib.pyplot as plt
x = target
y = tpred
plt.figure(figsize=(8, 6))
plt.scatter(x, y, s=10, c='blue', alpha=0.5)
plt.xlabel('Actual Y')
plt.ylabel('Y-OLS')
plt.title('OLS Prediction vs Target')
plt.grid(True)
plt.show()
plt.savefig('plot.png')

## Exercise 2 - Ridge Linear Regression

Recall that ridge regression is identical to OLS but with a L2 penalty over the weights:

$L(y,\hat{y})=\sum_{i=1}^{i=N}{(y^{(i)}-\hat{y}^{(i)})^2} + \lambda \left\Vert w \right\Vert_2^2$

where $y^{(i)}$ is the **true** value and $\hat{y}^{(i)}$ is the **predicted** value of the $i_{th}$ example, and $N$ is the number of examples

* Show, by differentiating the above loss, that the analytical solution is $w_{Ridge}=(X^TX+\lambda I)^{-1}X^Ty$
* Change `OrdinaryLinearRegression` and `OrdinaryLinearRegressionGradientDescent` classes to work also for ridge regression. Either add a parameter, or use inheritance.
* **Bonus: Noise as a regularizer**: Show that OLS (ordinary least square), if one adds multiplicative noise to the features the **average** solution for $W$ is equivalent to Ridge regression. In other words, if $X'= X*G$ where $G$ is an uncorrelated noise with variance $\sigma$ and mean 1, then solving for $X'$ with OLS is like solving Ridge for $X$. What is the interpretation?



In [ ]:
class RidgeLs(Ols):
  def __init__(self, ridge_lambda, *wargs, **kwargs):
    super(RidgeLs,self).__init__(*wargs, **kwargs)
    self.ridge_lambda = ridge_lambda

  def _fit(self, X, Y):
    #Closed form of ridge regression
    pass

### Use scikitlearn implementation for OLS, Ridge and Lasso